In [ ]:
import os
import trimesh
import trimesh.transformations as TF
from trimesh.proximity import closest_point
import torch
import numpy as np
import random
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from scipy import spatial

import meshlabxml as mlx
from tqdm import trange
import time

In [ ]:
# import Render
import Render_opencv as Render
Render.extIOR, Render.intIOR = 1.0, 1.5
res=512
Float = torch.float64
# Float = torch.float32
device='cuda'
Render.res = res
Render.device = device
Render.Float = Float

In [ ]:
scene =  Render.Scene("/root/workspace/data/mouse_vh.ply")


In [ ]:
alltime = 0
for _ in range(1000):
    start = time.time()
    scene.init_VN()
    alltime += time.time() - start
alltime

In [ ]:
%%capture
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
fig = plt.figure()
im = plt.imshow(gif[0], animated=True)
def update_fig(i):
    im.set_array(gif[i])
    return im,
anim = animation.FuncAnimation(fig, update_fig, frames=len(gif), interval=50, blit=True)


In [ ]:
HTML(anim.to_jshtml())

In [ ]:
# Render.extIOR, Render.intIOR = 1.15, 1.0
Render.extIOR, Render.intIOR = 1.5, 1.0

def render_target(scene:Render.Scene, deg):
    #俯视
    M=TF.rotation_matrix(-np.pi/180*0, [1,0,0])
    angle = TF.euler_from_matrix(TF.rotation_matrix(np.pi/180*deg, [0,1,0]) @ M)
    # scene.set_camera(fov=(60,60), distance = 1.0, center=(0,0.0,0), angles=angle)
    # scene.set_camera(fov=(60,60), distance = 1.1, center=(0,0.0,0), angles=angle)
    scene.set_camera(fov=(60,60), distance = 1.3, center=(0,0.0,0), angles=angle)
    camera_M = scene.camera_M()
    origin, ray_dir = scene.generate_ray()
    target, twice_mask = scene.render_transparent(origin, ray_dir)
    mask = scene.mask(origin, ray_dir)

    M = mask.view((res,res,1)).cpu().numpy()
    M = M.astype(np.uint8)
    bound = 2
    dist= (cv2.distanceTransform(M, cv2.DIST_L2, 0)-1).clip(0,bound)\
     - (cv2.distanceTransform(1-M, cv2.DIST_L2, 0)-1).clip(0,bound)
    soft_mask = (dist + bound) / (2*bound)
    soft_mask = torch.tensor(soft_mask, dtype=Float, device=device)

    return target, twice_mask, soft_mask, origin, ray_dir, camera_M

# Target = Render.Scene("/root/workspace/data/kitten.obj")
Target = Render.Scene("/root/workspace/data/bunny.ply")
Views = []
for deg in range(-90,90,15):
    Views.append(render_target(Target, deg))
# # for deg in [-30,-15,0,15,30]:
for deg in range(-90,90,15):
    Views.append(render_target(Target, deg+180))


In [ ]:
Render.torch2pil(Views[6][2])

In [ ]:

target, twice_mask, mask, origin, ray_dir, camera_M  = Views[6]
silhouette_edge = Target.silhouette_edge(origin[0])
index, output = Target.primary_visibility(silhouette_edge, camera_M, origin[0], detach_depth=True)
mask.view((res,res))[index[:,0],index[:,1]]
# mask.view((res,res))[index[:,0],index[:,1]] = 2
# Render.torch2pil(mask)

In [ ]:

# kitten_vh = Render.Scene("/root/workspace/DR/kitten_remesh.ply")
# kitten_vh = Render.Scene("/root/workspace/data/kitten_vh_sim.ply")
# kitten_vh = Render.Scene("/root/workspace/data/kitten_vh_sub.ply")
# kitten_vh = Render.Scene("/root/workspace/data/bunny_vh_sim.ply")
kitten_vh = Render.Scene("/root/workspace/data/bunny_vh_sub.ply")
# kitten_vh = Render.Scene("/root/workspace/DR/result/bunny_optim_remesh.ply")
vh_vertices = kitten_vh.vertices
vh_normals = torch.tensor(kitten_vh.mesh.vertex_normals, dtype=Render.Float, device=device)
vh_faces = torch.tensor(kitten_vh.mesh.faces, device=device)
parameter = torch.zeros(vh_vertices.shape, dtype=Float, requires_grad=True, device=device)
def limit_hook(grad):
    grad[grad>1]=1
    grad[grad<-1]=-1
    return grad
parameter.register_hook(limit_hook)
# opt = torch.optim.Adam([parameter], lr=.0003)
# opt = torch.optim.Adam([parameter], lr=.0003)
# opt = torch.optim.Adam([parameter], lr=.001)
# opt = torch.optim.SGD([parameter], lr=.001, momentum = 0.9, nesterov =True)
opt = torch.optim.SGD([parameter], lr=.001, momentum = 0.9, nesterov =True)

# e1 = kitten_vh.mesh.vertices[kitten_vh.mesh.edges[:,0]]
# e2 = kitten_vh.mesh.vertices[kitten_vh.mesh.edges[:,1]]
# mean_len = np.linalg.norm(e1-e2, axis=1).mean()

dist = []
_=Target.mesh.export("/dev/shm/DR/target.ply")

In [ ]:
# opt = torch.optim.SGD([parameter], lr=.0001, momentum = 0.9, nesterov =True)
# Render.extIOR, Render.intIOR = 1.45, 1.0
Render.extIOR, Render.intIOR = 1.5, 1.0

def area_var(vertices):
    v0=vertices[vh_faces[:,0]]
    v1=vertices[vh_faces[:,1]]
    v2=vertices[vh_faces[:,2]]
    edge1 = v1-v0
    edge2 = v2-v0
    area =torch.cross(edge1,edge2).norm(dim=1)
    area_ave = area.mean().detach()
    area_var = ((area-area_ave)/area_ave).pow(2).mean()
    return area_var
def mean_hausd(mesh):
    # closest, distance, triangle_id = closest_point(Target.mesh, vertices)
    # distance = distance*9.7*2
    # return distance.mean()
    tmppath = "/dev/shm/DR/temp.ply"
    mesh.export(tmppath)
    logpath = "/dev/shm/DR/hausd_log"
    os.system('rm '+logpath)
    cmd = "DISPLAY=:1 meshlabserver -i " + tmppath + " /dev/shm/DR/target.ply -s /root/workspace/data/hausd.mlx -l " + logpath
    assert (os.system(cmd)==0)
    dist = mlx.compute.parse_hausdorff(logpath)['mean_distance']
    return dist

for it in range(1):
    dist.append(mean_hausd(kitten_vh.mesh))
    for i in trange(400):
        # V_index = random.randint(0, len(Views)-1)
        V_index = 6
        target, twice_mask, mask, origin, ray_dir, camera_M = Views[V_index]
        # Zero out gradients before each iteration
        opt.zero_grad()

        # vertices = vh_vertices + parameter * vh_normals
        vertices = vh_vertices + parameter
        kitten_vh.update_verticex(vertices)

        render_img, render_twice_mask = kitten_vh.render_transparent(origin, ray_dir)
        twice_mask = (twice_mask * render_twice_mask)
        # loss = 1e3*(render_img[twice_mask]-target[twice_mask]).pow(2).mean()
        loss = 1e2*(render_img[twice_mask]-target[twice_mask]).pow(2).mean()

        silhouette_edge = kitten_vh.silhouette_edge(origin[0])
        index, output = kitten_vh.primary_visibility(silhouette_edge, camera_M, origin[0], detach_depth=True)
        vh_loss = 1e2 * (mask.view((res,res))[index[:,0],index[:,1]] - output).abs().mean()

        # rough = 0 * parameter.abs().mean()

        laplac = vertices - kitten_vh.weightM.mm(vertices) 
        sm_loss = 50*torch.norm(laplac/0.02, dim=1).pow(2).mean()    
        # sm_loss = 1*torch.norm(laplac/0.02, dim=1).pow(2).mean()    
        # sm_loss = torch.norm(laplac, dim=1).mean()
        area_loss = 1*area_var(vertices)
        # area_loss = 0*area_var(vertices)

        # (loss+sm_loss+vh_loss+area_loss).backward()
        vh_loss.backward()
        # if it % 200 == 0:
        #     # dist.append(mean_hausd(vertices.detach().cpu().numpy()))
        #     dist.append(mean_hausd(kitten_vh.mesh))


        # Optimizer: take a gradient step
        opt.step()

        # vertices_detach = vh_vertices + parameter.detach()
        # laplac = vertices_detach - kitten_vh.weightM.mm(vertices_detach) 
        # vh_vertices -= 0.3*laplac
        # vertices_detach = vh_vertices + parameter.detach()
        # laplac = vertices_detach - kitten_vh.weightM.mm(vertices_detach) 
        # vh_vertices += 0.3*laplac
        print('Iteration %03i: dist=%g error=%g sm=%g vh=%g grad=%g' % (it, dist[-1], loss, sm_loss, vh_loss, parameter.grad.abs().max()))

    # imageio.imsave("result/disp_{}.png".format(it), parameter.detach().cpu())

# _=mesh_vh.export("kitten_optim.ply")


In [ ]:
index.shape

In [ ]:
mask.view((res,res))[index[:,0],index[:,1]]

In [ ]:
# _=kitten_vh.mesh.export("kitten_optim2.ply")
_=kitten_vh.mesh.export("result/bunny_debug.ply")

In [ ]:
plt.plot(range(len(dist)),dist)
#ior 1.5

In [ ]:
plt.plot(range(len(dist)),dist)
#ior 1.45 -> 1.5

In [ ]:
plt.plot(range(len(dist)),dist)
#area loss 2

In [ ]:
plt.plot(range(len(dist)),dist)
#area loss 0

In [ ]:
dist[-1]*8.4

In [ ]:
# mesh = Render.Scene("/root/workspace/data/bunny_vh_sub.ply")
# closest, distance, triangle_id = closest_point(Target.mesh, mesh.mesh.vertices)
closest, distance, triangle_id = closest_point(Target.mesh, kitten_vh.mesh.vertices)
cmap = plt.cm.get_cmap('jet')
# distance = distance*9.7*2
distance = distance*8.4
color = distance.copy()
max_dist = 0.4
# max_dist = 0.75
color[color>max_dist]=max_dist
# mesh.mesh.visual.vertex_colors = cmap(color/max_dist)
kitten_vh.mesh.visual.vertex_colors = cmap(color/max_dist)
distance.mean()

In [ ]:
_=mesh.mesh.export("/root/workspace/data/bunny_vh_sub.ply")


In [ ]:
from trimesh.sample import sample_surface_even
sample,face_ind = sample_surface_even(kitten_vh.mesh, int(10e4))
closest, distance, triangle_id = closest_point(Target.mesh, sample)
distance = distance
distance.mean()

In [ ]:
v0 = kitten_vh.mesh.vertices[kitten_vh.mesh.faces[:,0]]
v1 = kitten_vh.mesh.vertices[kitten_vh.mesh.faces[:,1]]
v2 = kitten_vh.mesh.vertices[kitten_vh.mesh.faces[:,2]]
closest, distance0, triangle_id = closest_point(Target.mesh, v0)
closest, distance1, triangle_id = closest_point(Target.mesh, v1)
closest, distance2, triangle_id = closest_point(Target.mesh, v2)
distance = (distance0+distance1+distance2) / 3
(distance * kitten_vh.mesh.area_faces).sum() / kitten_vh.mesh.area_faces.sum() 

In [ ]:
kitten_vh.mesh.faces.shape

In [ ]:
kitten_vh.mesh.area_faces.shape

In [ ]:
kitten_vh.mesh.show()

In [ ]:
kitten_vh.mesh.extents

In [ ]:
# mesh = Render.Scene("/root/workspace/data/kitten_vh_sub.ply")
mesh = Render.Scene("/root/workspace/DR/bunny_optim_remesh.ply")
# point = mesh.mesh.vertices
# point[point[:,2]<-0.5, 2] =  -0.5
closest, distance, triangle_id = closest_point(Target.mesh, mesh.mesh.vertices)
cmap = plt.cm.get_cmap('jet')
# distance = distance*9.7*2
distance = distance*8.4*2
color = distance.copy()
# max_dist = 0.1
max_dist = distance.max()
color[color>max_dist]=max_dist
# mesh.mesh.visual.vertex_colors = cmap(color/max_dist)
distance.max()


In [ ]:
mask=mesh.mesh.vertices[:,1]>-0.48
distance[mask].mean()

In [ ]:
mesh.mesh.show()

In [ ]:
_=mesh.mesh.export("/root/workspace/DR/bunny_optim2.ply")